# Simulation Monte Carlo de la méthode d'Ising appliquer à la compliance des agents vis-à-vis des taxes

## Introduction

Dans ce programme, j'utilise la méthode de Monte Carlo pour simuler les propriétés ferromagnétiques d'un réseau bidimensionnel de spins en utilisant Python. Les spins de ce réseau correspondent à des agents qui tax-compliant ou non. En utilisant la méthode de Monte Carlo, où un réseau initial de spins orientés de manière aléatoire est retourné jusqu'à ce que le système atteigne un équilibre. La modélisation actuelle permet de faire les simulations à plusieurs températures.


## Background

Dans son sens le plus élémentaire, il modélise l'interaction entre les spins dans un matériau ferromagnétique.  Pour un système comportant $N$ sites qui contiennent un moment magnétique de spin, chaque site $S_ij$ peut avoir l'une des deux orientations possibles, $S_{ij} = 1$ (spin up/ agent compliant) ou $S_{ij} = -1$ (spin down/agent fraudeur).  Dans une configuration de réseau bidimensionnel, chaque site de spin a quatre voisins les plus proches (au-dessus, au-dessous, à gauche et à droite) et le modèle d'Ising simplifie l'analyse du ferromagnétique en ne prenant en compte que les interactions des spins directement voisins et en prenant un $\alpha$ = 0.

## La méthode de Monte Carlo

La méthode de Monte Carlo pour analyser le modèle d'Ising est assez simple. Le système initial du réseau est dans un état de configurations de spin aléatoires. Nous choisissons ensuite un site sur le réseau de façon aléatoire et calculons le changement d'énergie de la nouvelle configuration. Puisque nous ne sommes concernés que par les interactions entre voisins les plus proches, nous n'avons pas besoin de recalculer l'énergie de l'ensemble du système. Nous calculons simplement le changement d'énergie puisque nous savons quelles valeurs possibles il doit prendre pour un retournement de spin donné.


Si le changement d'énergie est inférieur à zéro, nous acceptons la nouvelle configuration de spin. Cela est logique puisque le système aura tendance à s'équilibrer vers l'état d'énergie le plus bas. Si le changement d'énergie est supérieur à zéro, nous l'acceptons avec une probabilité. Puisqu'il y a toujours une chance que le système passe à l'état d'énergie supérieur (même si c'est improbable), nous devons en tenir compte. La probabilité que nous utilisons est écrite ci-dessous :

$p=\frac{1}{1+e^{\frac{-{\Delta{E}}}{{k_B}{T}}}}$

Où ${\Delta{E}}$ est le changement d'énergie du système après le retournement du spin, $T$ est la température du système et $k_B$ est la constante de Boltzmann. Dans cette simulation, la constante de Boltzmann est fixée à 1. Afin d'utiliser cette fonction, nous générons un nombre aléatoire $x$ entre 0 et 1. Si nous retournons un spin et qu'il finit par avoir un changement positif d'énergie, nous acceptons cette nouvelle configuration uniquement si $p>x$.  Si $p$ est inférieur à $x$, nous rejetons le retournement et conservons l'ancienne configuration.

## Implementation dans Python

In [11]:
from numpy import *
from random import *
from tkinter import *

The user is allowed to input the number of sites on each row of the two dimensional lattice, the width of each cell in the interface simulation and the number of Monte Carlo steps that are taken in the simulation.  Because the screen on the computer is only so large, I find that keeping the number of sites between 50 and 75 is ideal.  Similarly the width of each cell should be reasonably set.  The width is given in pixels and a width between 10 and 15 is good. Lastly the number of steps in the simulation has one main visual effect on the interface.  If the number of Monte Carlo steps are small then the spins will flip quickly on the interface simulation.  If the number of steps is large then the spins will flip more slowly on the interface.

L'utilisateur peut saisir le nombre de sites sur chaque rangée du réseau bidimensionnel, la largeur de chaque cellule dans la simulation de l'interface et le nombre de pas de Monte Carlo effectués dans la simulation. Il faut adapter les deux premières variables en fonction de la taille de son écran. Si le nombre d'étapes de Monte Carlo est faible, les spins se retourneront rapidement sur la simulation de l'interface. Si le nombre d'étapes est élevé, les spins se retourneront plus lentement sur l'interface.
Valeurs recommandés :
n = 50
w = 10
steps = 10

In [12]:
#On définit les commandes
kb=1 #.38*10**-23 #boltzmann

#user inputs
n = int(input('Input the number of sites in each row: '))
w = int(input('Input the cell width of each site in pixels:'))
steps = int(input('Input the number of Monte Carlo steps:'))

L'implémentation de la fonction d'énergie en python nécessite la prise en compte des bords de la grille.  Puisque la simulation prend en compte le changement d'énergie en fonction des spins voisins, nous devons envisager les cas où le site choisi sur le réseau n'a que deux voisins les plus proches au lieu de quatre.

In [13]:
#On définit le changement d'énergie
def deltaE(i,j):
    if i==0:
        left=lattice[sites-1,j] 
    else:
        left=lattice[i-1,j]
    if i==sites-1:
        right=lattice[0,j]
    else:
        right=lattice[i+1,j]
    if j==0:
        above=lattice[i,sites-1]
    else:
        above=lattice[i,j-1]
    if j==sites-1:
        below=lattice[i,0]
    else:
        below=lattice[i,j+1]
    return 2.0 * lattice[i,j] * (left+right+above+below)

La fonction pixel ci-dessous regarde simplement un site et lui attribue une des deux couleurs.  Si le spin de ce site est égal à 1 (spin up), la couleur attribuée est le vert.  Si le spin est égal à -1 (spin down), la couleur est grise.  La dernière ligne de code dans le bloc ci-dessous colore les pixels correspondants sur l'interface.

In [14]:
#On définit la fonction qui assigne les couleurs aux pixels
def pixel(i,j):
    if lattice[i,j]==1:
        pixel = "#2a623d" #vert = spin up / agent compliant
    else:
        pixel = "#aaaaaa" #gris = spin down / agent fraudeur
        
    image.put(pixel, to=(i*sitewidth,j*sitewidth,(i+1)*sitewidth,(j+1)*sitewidth))#On colorit les pixels correspondants


Ci-dessous se trouve la boucle principale de Monte Carlo. Le code ne passe par cette boucle que si l'interface exécute la simulation. La température peut être ajustée sur l'interface (voir le code plus bas) et cette boucle obtient d'abord cette température. Ensuite, la boucle choisit un site au hasard, elle calcule le changement d'énergie en appelant la fonction d'énergie et si ce changement est inférieur ou égal à zéro, elle retourne le spin. S'il est supérieur à zéro, il renverse le spin avec une probabilité. La boucle ajuste ensuite la couleur sur l'interface.

In [15]:
def montecarlo():
    if running:
        T = tempscale.get()#On récupère la température issue de la barre glissante dans l'interface
        for step in range(steps):#On effectue la boucle sur le nombre de steps définies au début
            i = int(random()*sites)#On choisit une ligne et une colonne au hasard
            j = int(random()*sites)
            x=round(uniform(0.,1.),10)#On choisit un nombre aléatoire entre 0 et 1
            p=1/(1+exp(-deltaE(i,j)/(kb*T)))
            if deltaE(i,j)<=0:#On flip le spin si le changement d'énergie est inférieur à 0
                lattice[i,j] = -lattice[i,j]
            elif p>x:#Si l'énergie est supérieure à 0, on flip le spin seulement si p > x
                lattice[i,j] = -lattice[i,j]
                pixel(i, j)#On appelle la fonction pixel pour effectuer la coloration
    window.after(1,montecarlo)

In [16]:
sites = n#nombre de spin sur chaque colonne
#On crée une grille de spin en 2D
lattice = zeros((sites, sites), int)
sitewidth=w# On détermine la largeur de chaque spin
canvaswidth=sites * sitewidth# On détermine la largeur de notre canva
running=False

window=Tk()
window.title("Simulation of the Ising Model")
window.geometry('+50+50')

#On crée le canva pour simulation les transitions des spins
canvas = Canvas(window, width=canvaswidth, height=canvaswidth)
canvas.pack()
image = PhotoImage(width=canvaswidth, height=canvaswidth)
canvas.create_image((3, 3), image=image, anchor="nw", state="normal")

1

In [17]:
#On définit la fonction qui démarre et arrête la simulation
#On crée un bouton qui appelle cette fonction
def startandstop():
    global running
    running = not running
    if running:
        startbutton.config(text="stop")
    else:
        startbutton.config(text="start")

On a là les commands pour tkinter

In [18]:
controlFrame = Frame(window)#On crée la frame
controlFrame.pack()
tempscale = Scale(controlFrame, from_=0.01, to=10., resolution=0.01, length=300, orient="horizontal")
tempscale.pack(side="left")
tempscale.set(2.3)# On définit ici la température de départ
#Il serait judicieux ici de prendre le température de Curie associé à notre système
#Cependant, nous ne l'avons pas
templabel = Label(controlFrame, text="temperature:")
templabel.pack(side="left")
spacer = Frame(controlFrame, width=40)
spacer.pack(side="left")
startbutton = Button(controlFrame, text="start", width=8, command=startandstop)
startbutton.pack(side="left")

Les derniers blocs contiennent les commandes réelles qui exécutent le code. Tout d'abord, un treillis aléatoire de spins est créé. Ensuite, la fonction Monte Carlo est exécutée, suivie par la fonction d'interface utilisateur.

In [19]:
spins=[-1,1]
#On crée une grille aléatoire de spins
for i in range(sites):
    for j in range(sites):
        lattice[i,j]=choice(spins)
        pixel(i,j)#On appelle la fonction pixel pour colorier le canva

In [20]:
montecarlo()#On lance la boucle monte Carlo
window.mainloop()#On lance l'interface visuelle

C:\Users\Antoine\AppData\Local\Temp\ipykernel_1444\4112245994.py:9: RuntimeWarning: overflow encountered in exp
  p=1/(1+exp(-deltaE(i,j)/(kb*T)))


## Resultats

Pour obtenir des résultats vraiment pertinent sur les différences de comportement en fonction de la température, il faudrait laisser tourner l'algorithme jusqu'à ce qu'ils réalisent des milliards de changements, ce qui nous prendrait plusieurs jours. Or nous ne pouvons pas réaliser cela. Et donc même si ce n'est pas très scientifique nous devons faire confiance au papier recherche.